In [39]:
from scipy.linalg import dft, circulant
from numpy import diag, eye
from numpy.linalg import inv
import numpy as np
from typing import Callable
from functools import reduce
import operator

In [2]:
dft(4)

array([[ 1.0000000e+00+0.0000000e+00j,  1.0000000e+00+0.0000000e+00j,
         1.0000000e+00+0.0000000e+00j,  1.0000000e+00+0.0000000e+00j],
       [ 1.0000000e+00+0.0000000e+00j,  6.1232340e-17-1.0000000e+00j,
        -1.0000000e+00-1.2246468e-16j, -1.8369702e-16+1.0000000e+00j],
       [ 1.0000000e+00+0.0000000e+00j, -1.0000000e+00-1.2246468e-16j,
         1.0000000e+00+2.4492936e-16j, -1.0000000e+00-3.6739404e-16j],
       [ 1.0000000e+00+0.0000000e+00j, -1.8369702e-16+1.0000000e+00j,
        -1.0000000e+00-3.6739404e-16j,  5.5109106e-16-1.0000000e+00j]])

In [5]:
circulant([1,2,0,0,2])

array([[1, 2, 0, 0, 2],
       [2, 1, 2, 0, 0],
       [0, 2, 1, 2, 0],
       [0, 0, 2, 1, 2],
       [2, 0, 0, 2, 1]])

In [9]:
n = 5
eye(n, k=1)

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0.]])

In [23]:
def squared_exponential_covariance_function(length: float) -> Callable:
    def k(x):
        return np.exp(-1/2 * (x/length)**2 )
    return k

In [ ]:
def get_covariance_matrix(n: int, k: Callable):
    

In [25]:
k = squared_exponential_covariance_function(1)

In [20]:
np.set_printoptions(formatter={'float': "{0:0.3f}".format})

In [21]:
I, J = np.meshgrid(range(n), range(n), indexing='ij')

In [26]:
k = squared_exponential_covariance_function(1)
cov = k(I - J)
cov

array([[1.000, 0.607, 0.135, 0.011, 0.000],
       [0.607, 1.000, 0.607, 0.135, 0.011],
       [0.135, 0.607, 1.000, 0.607, 0.135],
       [0.011, 0.135, 0.607, 1.000, 0.607],
       [0.000, 0.011, 0.135, 0.607, 1.000]])

In [27]:
k = squared_exponential_covariance_function(1/2)
cov = k(I - J)
cov

array([[1.000, 0.135, 0.000, 0.000, 0.000],
       [0.135, 1.000, 0.135, 0.000, 0.000],
       [0.000, 0.135, 1.000, 0.135, 0.000],
       [0.000, 0.000, 0.135, 1.000, 0.135],
       [0.000, 0.000, 0.000, 0.135, 1.000]])

In [31]:
inv(cov)

array([[1.019, -0.140, 0.019, -0.003, 0.000],
       [-0.140, 1.038, -0.143, 0.019, -0.003],
       [0.019, -0.143, 1.039, -0.143, 0.019],
       [-0.003, 0.019, -0.143, 1.038, -0.140],
       [0.000, -0.003, 0.019, -0.140, 1.019]])

# Energy matrices 
## Non-circular case

In [57]:
def diff_mat(n, k=None):
    if k is None:
        return eye(n-1, n, dtype=int) - eye(n-1, n, k=1, dtype=int)
    
    if k == 0:
        return eye(n, dtype=int)
    
    return reduce(operator.matmul, [diff_mat(n-i) for i in reversed(range(k))])

diff_mat(n)

array([[ 1, -1,  0,  0,  0],
       [ 0,  1, -1,  0,  0],
       [ 0,  0,  1, -1,  0],
       [ 0,  0,  0,  1, -1]])

In [62]:
for k in range(n):
    print(f"\nk={k} \n{diff_mat(n, k)}")


k=0 
[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]

k=1 
[[ 1 -1  0  0  0]
 [ 0  1 -1  0  0]
 [ 0  0  1 -1  0]
 [ 0  0  0  1 -1]]

k=2 
[[ 1 -2  1  0  0]
 [ 0  1 -2  1  0]
 [ 0  0  1 -2  1]]

k=3 
[[ 1 -3  3 -1  0]
 [ 0  1 -3  3 -1]]

k=4 
[[ 1 -4  6 -4  1]]


In [64]:
for k in range(n):
    d_k = diff_mat(n, k)
    m_k = d_k.T @ d_k
    print(f"\nk={k} \n{m_k}")


k=0 
[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]

k=1 
[[ 1 -1  0  0  0]
 [-1  2 -1  0  0]
 [ 0 -1  2 -1  0]
 [ 0  0 -1  2 -1]
 [ 0  0  0 -1  1]]

k=2 
[[ 1 -2  1  0  0]
 [-2  5 -4  1  0]
 [ 1 -4  6 -4  1]
 [ 0  1 -4  5 -2]
 [ 0  0  1 -2  1]]

k=3 
[[  1  -3   3  -1   0]
 [ -3  10 -12   6  -1]
 [  3 -12  18 -12   3]
 [ -1   6 -12  10  -3]
 [  0  -1   3  -3   1]]

k=4 
[[  1  -4   6  -4   1]
 [ -4  16 -24  16  -4]
 [  6 -24  36 -24   6]
 [ -4  16 -24  16  -4]
 [  1  -4   6  -4   1]]


In [89]:
coeffs = [0.1, 0, 2]
K = sum(c * diff_mat(n, k).T @ diff_mat(n, k) for k, c in enumerate(coeffs))
K, inv(K)

(array([[2.100, -4.000, 2.000, 0.000, 0.000],
        [-4.000, 10.100, -8.000, 2.000, 0.000],
        [2.000, -8.000, 12.100, -8.000, 2.000],
        [0.000, 2.000, -8.000, 10.100, -4.000],
        [0.000, 0.000, 2.000, -4.000, 2.100]]),
 array([[6.170, 3.905, 1.831, -0.056, -1.850],
        [3.905, 3.083, 2.065, 1.003, -0.056],
        [1.831, 2.065, 2.208, 2.065, 1.831],
        [-0.056, 1.003, 2.065, 3.083, 3.905],
        [-1.850, -0.056, 1.831, 3.905, 6.170]]))

## Circular case

In [78]:
def cdiff_mat(n, k=None):
    if k is None:
        v = np.zeros(n)
        v[0] = 1
        v[1] = -1
        return circulant(v).astype(int).T
    
    if k == 0:
        return eye(n, dtype=int)
    
    cdiff_1 = cdiff_mat(n)
    return reduce(operator.matmul, [cdiff_1 for i in reversed(range(k))])

cdiff_mat(n)

array([[ 1, -1,  0,  0,  0],
       [ 0,  1, -1,  0,  0],
       [ 0,  0,  1, -1,  0],
       [ 0,  0,  0,  1, -1],
       [-1,  0,  0,  0,  1]])

In [79]:
for k in range(n):
    print(f"\nk={k} \n{cdiff_mat(n, k)}")


k=0 
[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]

k=1 
[[ 1 -1  0  0  0]
 [ 0  1 -1  0  0]
 [ 0  0  1 -1  0]
 [ 0  0  0  1 -1]
 [-1  0  0  0  1]]

k=2 
[[ 1 -2  1  0  0]
 [ 0  1 -2  1  0]
 [ 0  0  1 -2  1]
 [ 1  0  0  1 -2]
 [-2  1  0  0  1]]

k=3 
[[ 1 -3  3 -1  0]
 [ 0  1 -3  3 -1]
 [-1  0  1 -3  3]
 [ 3 -1  0  1 -3]
 [-3  3 -1  0  1]]

k=4 
[[ 1 -4  6 -4  1]
 [ 1  1 -4  6 -4]
 [-4  1  1 -4  6]
 [ 6 -4  1  1 -4]
 [-4  6 -4  1  1]]


In [81]:
for k in range(n):
    d_k = cdiff_mat(n, k)
    m_k = d_k.T @ d_k
    print(f"\nk={k} \n{m_k}")


k=0 
[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]

k=1 
[[ 2 -1  0  0 -1]
 [-1  2 -1  0  0]
 [ 0 -1  2 -1  0]
 [ 0  0 -1  2 -1]
 [-1  0  0 -1  2]]

k=2 
[[ 6 -4  1  1 -4]
 [-4  6 -4  1  1]
 [ 1 -4  6 -4  1]
 [ 1  1 -4  6 -4]
 [-4  1  1 -4  6]]

k=3 
[[ 20 -15   5   5 -15]
 [-15  20 -15   5   5]
 [  5 -15  20 -15   5]
 [  5   5 -15  20 -15]
 [-15   5   5 -15  20]]

k=4 
[[ 70 -55  20  20 -55]
 [-55  70 -55  20  20]
 [ 20 -55  70 -55  20]
 [ 20  20 -55  70 -55]
 [-55  20  20 -55  70]]


In [83]:
coeffs = [1, 1, 1]
K = sum(c * cdiff_mat(n, k).T @ cdiff_mat(n, k) for k, c in enumerate(coeffs))
K, inv(K)

(array([[ 9, -5,  1,  1, -5],
        [-5,  9, -5,  1,  1],
        [ 1, -5,  9, -5,  1],
        [ 1,  1, -5,  9, -5],
        [-5,  1,  1, -5,  9]]),
 array([[0.316, 0.211, 0.132, 0.132, 0.211],
        [0.211, 0.316, 0.211, 0.132, 0.132],
        [0.132, 0.211, 0.316, 0.211, 0.132],
        [0.132, 0.132, 0.211, 0.316, 0.211],
        [0.211, 0.132, 0.132, 0.211, 0.316]]))